## 📦 Coding-Labs Dateipacker

Mit diesem Notebook können Sie Notebooks, Dateien und Ordner in ein Installer-Notebook zusammenfassen, welches dann z. B. per E-Mail verteilt werden kann. Die Schüler*innen laden das Notebook in ihren Account hoch und führen den darin befindlichen Code einmal aus. Damit wird die originale Datei- und Ordnerstruktur wiederhergestellt.

**Wichtig:** Vor dem Ausführen der folgenden Zelle muss die Zelle unter ***Diese Zelle hier zuerst ausführen*** ausgeführt werden, um notwendige Funktionen zu laden. 

Geben Sie nun in der folgenden Zelle Quellordner und Zielnotebook an und führen die Zelle dann aus. Dadurch wird das Zielnotebook erstellt, welches Sie dann verteilen können.

**Hinweis:** Es kann einen Moment dauern, bis das erstellte Notebook links im Dateimanager erscheint.

In [ ]:
dateien_ordner = " ...  "  ## Bitte geben Sie hier den Ordner an, in dem sich die zu packenden Dateien befinden.
installer_name = " ...  "  ## Hier muss der Name des Installations-Notebooks hin, welches Sie dann verteilen können.
   
import subprocess
subprocess.Popen( 'tar -zcvf data.tar "' + dateien_ordner + '"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE )
! cat data.tar | base64 >data.txt
create_installer()
! rm data.tar; rm data.txt
print("Fertig.")

So sieht die Titelzeile vom Installations-Notebook aus, wie es die Schüler*innen sehen.

---

## Datei-Installer
Führen Sie die Python-Zelle aus, um den Ordner mit dem Kurs im Wurzelverzeichnis zu erstellen. 

***Hinweis:*** Es kann einen Moment dauern, bis der neu erstellte Ordner links im Dateimanager erscheint. Danach bitte einmal abmelden und wieder anmelden. Das Installationsnotebook kann dann gelöscht werden.

---

## Diese Zelle hier zuerst ausführen:

In [ ]:
import json
this_notebook = "CL_dateipacker.ipynb"

def pattern_in_cell_source(json_cell, pattern):
    if len(json_cell["source"]) > 0:
        for line_index in range(len(json_cell["source"])):
            if json_cell["source"][line_index].startswith(pattern):
                return True                    
    return False
        
def extract_config_element(json_cell, pattern):
    for element in json_cell:
        if element.startswith(pattern):
            return element.split('=')[1].strip('\n "')

def create_jupyter_out(json_block):
    json_block_out = json_block.copy()
    json_block_out["cells"] = []
    for cell_index in range(len(json_block["cells"])):
        if pattern_in_cell_source(json_block["cells"][cell_index], "## Datei-Installer") or \
           pattern_in_cell_source(json_block["cells"][cell_index], "## Datei-Installer-Code"):
            json_block_out["cells"].append(json_block["cells"][cell_index])
    return json_block_out
         
def add_base64_segment(json_block):
    base64file = open("data.txt")
    base64_block=""
    for line in base64file:
        base64_block = base64_block + line
    json_block_mod = json_block.copy()
    for cell_index in range(len(json_block_mod["cells"])):
        if pattern_in_cell_source(json_block_mod["cells"][cell_index], "## Datei-Installer-Code"):
            json_block_mod["cells"][cell_index]["metadata"]["CL_base64_block"] = base64_block
    return json_block_mod

def add_code(json_block):
    add_code = "this_notebook = " + '"' + installer_name+'"\n'
    json_block_final = json_block.copy()
    for cell_index in range(len(json_block_final["cells"])):
        if pattern_in_cell_source(json_block_final["cells"][cell_index], "## Datei-Installer-Code"):
            json_block_final["cells"][cell_index]["source"] = [add_code] + json_block_final["cells"][cell_index]["source"]
    return json_block_final      

def create_installer():
    json_block = json.load(open(this_notebook))
    json_block_out = create_jupyter_out(json_block)
    json_block_mod = add_base64_segment(json_block_out)
    json_block_final = add_code(json_block_mod)
    oup = open(installer_name, "w", encoding="UTF-8")
    json_out = json.dumps(json_block_out, indent=1, ensure_ascii=False)
    oup.write(json_out)
    oup.close()



## Diese Zelle nicht ausführen!

In [ ]:
## Datei-Installer-Code

import json

def pattern_in_cell_source(json_cell, pattern):
    if len(json_cell["source"]) > 0:
        for line_index in range(len(json_cell["source"])):
            if json_cell["source"][line_index].startswith(pattern):
                return True                    
    return False

json_block = json.load(open(this_notebook))
for cell_index in range(len(json_block["cells"])):
    if pattern_in_cell_source(json_block["cells"][cell_index], "## Datei-Installer-Code"):
        base64_block = json_block["cells"][cell_index]["metadata"]["CL_base64_block"]
base64file = open("data_out.txt","w")
base64file.write(base64_block)
base64file.close()
! cat data_out.txt | base64 -d > data.tar
! tar -k -zxvf data.tar
! rm data_out.txt
! rm data.tar

<font color="#a0a0a0" >Autor: Joachim Kutzera | Version: 09. Juli, 2024 - © 2024 | Lizensiert unter CC BY-NC 4.0
   | codinglabs-projekt.de | infai.org
</font>